In [ ]:
from z3 import *

# Declare symbolic variables
src = Int('src')
dst = Int('dst')

# Define the matches function
def matches(srcLower, srcUpper, dstLower, dstUpper):
    return And(
        srcLower <= src, src <= srcUpper,
        dstLower <= dst, dst <= dstUpper
    )

# Define firewall1 and firewall2 as Boolean expressions
firewall1 = And(
    matches(0, 10, 20, 30),
    Not(matches(5, 10, 25, 30))
)

firewall2 = And(
    matches(1, 10, 20, 30),
    Not(matches(5, 10, 25, 30))
)

# Create the solver
s = Solver()

# Assert that firewall1 and firewall2 are not equivalent
s.add(Not(firewall1 == firewall2))

# Check satisfiability and print the model
if s.check() == sat:
    print("SAT")
    print(s.model())
else:
    print("UNSAT")

In [ ]:
'''
import ipaddress

class AddressRange:
    def __init__(self, low, high):
        self.low = ipaddress.IPv4Address(low)
        self.high = ipaddress.IPv4Address(high)

class AddressSet:
    def __init__(self, contains_all=False, ranges=None):
        self.contains_all = contains_all
        self.ranges = ranges or []

class PortRange:
    def __init__(self, low, high):
        self.low = int(low)
        self.high = int(high)

class PortSet:
    def __init__(self, contains_all=False, ranges=None):
        self.contains_all = contains_all
        self.ranges = ranges or []

class NetworkProtocol:
    def __init__(self, any_protocol=False, protocol_number=None):
        self.any = any_protocol
        self.protocol_number = protocol_number

class WindowsFirewallRule:
    def __init__(self, name, local_ports, remote_addresses, remote_ports, protocol, enabled, allow):
        self.name = name
        self.local_ports = local_ports
        self.remote_addresses = remote_addresses
        self.remote_ports = remote_ports
        self.protocol = protocol
        self.enabled = enabled
        self.allow = allow

def parse_address_set(text):
    text = text.strip()
    if text == "Any":
        return AddressSet(contains_all=True)
    
    ranges = []
    for part in text.split(','):
        part = part.strip()
        if '-' in part:
            low, high = part.split('-')
            ranges.append(AddressRange(low.strip(), high.strip()))
        else:
            ip = part.strip()
            ranges.append(AddressRange(ip, ip))
    
    return AddressSet(contains_all=False, ranges=ranges)

def parse_port_set(text):
    text = text.strip()
    if not text:
        raise ValueError("Port is empty")
    if text == "Any":
        return PortSet(contains_all=True)

    macros = {"RPC Endpoint Mapper", "RPC Dynamic Ports", "IPHTTPS", "Edge Traversal", "PlayTo Discovery"}
    if text in macros:
        raise ValueError(f"Unsupported port macro: {text}")

    ranges = []
    for part in text.split(','):
        part = part.strip()
        if '-' in part:
            low, high = part.split('-')
            ranges.append(PortRange(low.strip(), high.strip()))
        else:
            ranges.append(PortRange(part, part))
    
    return PortSet(contains_all=False, ranges=ranges)

def parse_protocol(text):
    text = text.strip()
    if text == "Any":
        return NetworkProtocol(any_protocol=True)
    
    try:
        protocol_number = int(text)
    except ValueError:
        proto_map = {"TCP": 6, "UDP": 17}
        if text.upper() not in proto_map:
            raise ValueError(f"Unknown protocol: {text}")
        protocol_number = proto_map[text.upper()]
    
    return NetworkProtocol(any_protocol=False, protocol_number=protocol_number)

def parse_rule_line(header_index, line, separator):
    fields = [f.strip() for f in line.split(separator)]
    
    return WindowsFirewallRule(
        name=fields[header_index["Name"]],
        local_ports=parse_port_set(fields[header_index["Local Port"]]),
        remote_addresses=parse_address_set(fields[header_index["Remote Address"]]),
        remote_ports=parse_port_set(fields[header_index["Remote Port"]]),
        protocol=parse_protocol(fields[header_index["Protocol"]]),
        enabled=fields[header_index["Enabled"]].strip() == "Yes",
        allow=fields[header_index["Action"]].strip() == "Allow"
    )

def parse_firewall_dump(text, separator='\t'):
    lines = text.strip().split('\n')
    headers = [h.strip() for h in lines[0].split(separator)]
    header_index = {h: i for i, h in enumerate(headers)}
    required = {"Name", "Enabled", "Action", "Local Port", "Remote Address", "Remote Port", "Protocol"}
    missing = required - header_index.keys()
    if missing:
        raise ValueError(f"Missing headers: {', '.join(missing)}")
    
    rules = []
    for i, line in enumerate(lines[1:]):
        if not line.strip():
            continue
        try:
            rule = parse_rule_line(header_index, line, separator)
            rules.append(rule)
        except Exception as e:
            print(f"Skipping line {i + 2}: {e}")
    
    return rules






from z3 import *

# Define symbolic traffic fields
src_ip = BitVec('src_ip', 32)
dst_ip = BitVec('dst_ip', 32)
src_port = BitVec('src_port', 16)
dst_port = BitVec('dst_port', 16)
protocol = BitVec('protocol', 8)


def ip_to_int(ip_str):
    import ipaddress
    return int(ipaddress.IPv4Address(ip_str))

def match_rule(rule):
    clauses = []

    # Protocol check
    if not rule.protocol.any:
        clauses.append(protocol == rule.protocol.protocol_number)

    # Remote address check
    if not rule.remote_addresses.contains_all:
        ip_clauses = []
        for r in rule.remote_addresses.ranges:
            low = ip_to_int(r.low)
            high = ip_to_int(r.high)
            ip_clauses.append(And(dst_ip >= low, dst_ip <= high))
        clauses.append(Or(*ip_clauses))

    # Remote port check
    if not rule.remote_ports.contains_all:
        port_clauses = []
        for r in rule.remote_ports.ranges:
            port_clauses.append(And(dst_port >= r.low, dst_port <= r.high))
        clauses.append(Or(*port_clauses))

    # Local port check
    if not rule.local_ports.contains_all:
        local_port_clauses = []
        for r in rule.local_ports.ranges:
            local_port_clauses.append(And(src_port >= r.low, src_port <= r.high))
        clauses.append(Or(*local_port_clauses))

    # Combine all rule checks
    return And(*clauses)

def build_firewall_policy(rules):
    for rule in rules:
        if not rule.enabled:
            continue
        match = match_rule(rule)
        yield (match, rule.allow)

def firewall_decision(rules):
    """
    Returns Z3 expression that determines if traffic is allowed by rule set.
    Follows first-match semantics.
    """
    policy = build_firewall_policy(rules)
    result = False
    for match, allow in policy:
        result = If(match, allow, result)
    return result

s = Solver()

# Define policies
decision1 = firewall_decision(rules1)
decision2 = firewall_decision(rules2)

# Ask if there's any traffic that is treated differently
s.add(decision1 != decision2)

if s.check() == sat:
    model = s.model()
    print("Found traffic that is treated differently:")
    print(f"src_port = {model[src_port]}")
    print(f"dst_port = {model[dst_port]}")
    print(f"dst_ip = {model[dst_ip]}")
    print(f"protocol = {model[protocol]}")
else:
    print("The two rule sets are equivalent.")

'''

In [1]:
# Python version of WindowsFirewallRuleParser.cs
# This assumes supporting classes like AddressSet, AddressRange, PortSet, PortRange, and NetworkProtocol are defined.

import ipaddress
import csv
from typing import List, Dict, Generator, Optional

class AddressRange:
    def __init__(self, low: str, high: str):
        self.low = ipaddress.IPv4Address(low)
        self.high = ipaddress.IPv4Address(high)

class AddressSet:
    def __init__(self, contains_all: bool = False, ranges: Optional[List[AddressRange]] = None):
        self.contains_all = contains_all
        self.ranges = ranges or []

class PortRange:
    def __init__(self, low: int, high: int):
        self.low = low
        self.high = high

class PortSet:
    def __init__(self, contains_all: bool = False, ranges: Optional[List[PortRange]] = None):
        self.contains_all = contains_all
        self.ranges = ranges or []

class NetworkProtocol:
    def __init__(self, any_protocol: bool = False, protocol_number: int = 0):
        self.any = any_protocol
        self.protocol_number = protocol_number

    @staticmethod
    def try_get_protocol_number(name: str) -> Optional[int]:
        protocol_map = {"TCP": 6, "UDP": 17, "ICMP": 1}
        return protocol_map.get(name.upper())

class WindowsFirewallRule:
    def __init__(self, name: str, remote_addresses: AddressSet, remote_ports: PortSet,
                 local_ports: PortSet, protocol: NetworkProtocol, enabled: bool, allow: bool):
        self.name = name
        self.remote_addresses = remote_addresses
        self.remote_ports = remote_ports
        self.local_ports = local_ports
        self.protocol = protocol
        self.enabled = enabled
        self.allow = allow

class WindowsFirewallRuleParser:
    REQUIRED_HEADERS = [
        "Name", "Enabled", "Action", "Local Port",
        "Remote Address", "Remote Port", "Protocol"
    ]

    @staticmethod
    def parse(text: str, separator: str) -> Generator[WindowsFirewallRule, None, None]:
        reader = csv.reader(text.strip().splitlines(), delimiter=separator)
        header_line = next(reader)
        header_index = WindowsFirewallRuleParser.parse_header(header_line)

        for i, line in enumerate(reader):
            try:
                yield WindowsFirewallRuleParser.parse_record(header_index, line)
            except Exception as e:
                print(f"Skipping line {i + 2} - {e}")

    @staticmethod
    def parse_header(header_line: List[str]) -> Dict[str, int]:
        header_index = {h.strip(): i for i, h in enumerate(header_line)}
        missing = [h for h in WindowsFirewallRuleParser.REQUIRED_HEADERS if h not in header_index]
        if missing:
            raise ValueError(f"Missing required headers: {', '.join(missing)}")
        return header_index

    @staticmethod
    def parse_record(header_index: Dict[str, int], record: List[str]) -> WindowsFirewallRule:
        def get(field: str) -> str:
            return record[header_index[field]].strip()

        return WindowsFirewallRule(
            name=get("Name"),
            remote_addresses=WindowsFirewallRuleParser.parse_address_set(get("Remote Address")),
            remote_ports=WindowsFirewallRuleParser.parse_port_set(get("Remote Port")),
            local_ports=WindowsFirewallRuleParser.parse_port_set(get("Local Port")),
            protocol=WindowsFirewallRuleParser.parse_network_protocol(get("Protocol")),
            enabled=(get("Enabled") == "Yes"),
            allow=(get("Action") == "Allow")
        )

    @staticmethod
    def parse_address_set(text: str) -> AddressSet:
        if text == "Any":
            return AddressSet(contains_all=True)

        ranges = []
        for part in text.split(','):
            part = part.strip()
            if '-' in part:
                low, high = part.split('-')
                ranges.append(AddressRange(low.strip(), high.strip()))
            else:
                ranges.append(AddressRange(part, part))
        return AddressSet(contains_all=False, ranges=ranges)

    @staticmethod
    def parse_port_set(text: str) -> PortSet:
        if not text:
            raise ValueError("Port is empty")

        if text == "Any":
            return PortSet(contains_all=True)

        unsupported_macros = {
            "RPC Endpoint Mapper", "RPC Dynamic Ports",
            "IPHTTPS", "Edge Traversal", "PlayTo Discovery"
        }
        if text in unsupported_macros:
            raise ValueError(f"Unsupported port macro: {text}")

        ranges = []
        for part in text.split(','):
            part = part.strip()
            if '-' in part:
                low, high = part.split('-')
                ranges.append(PortRange(int(low.strip()), int(high.strip())))
            else:
                val = int(part)
                ranges.append(PortRange(val, val))
        return PortSet(contains_all=False, ranges=ranges)

    @staticmethod
    def parse_network_protocol(text: str) -> NetworkProtocol:
        if text == "Any":
            return NetworkProtocol(any_protocol=True)

        protocol_number = NetworkProtocol.try_get_protocol_number(text)
        if protocol_number is None:
            protocol_number = int(text)

        return NetworkProtocol(any_protocol=False, protocol_number=protocol_number)

# Example of taking input and parsing
if __name__ == '__main__':
    import sys
    
    filename = input("Enter the firewall dump file path: ")
    # separator = input("Enter the column separator (e.g., ','): ")

    with open(filename, 'r') as f:
        text = f.read()

    print("Parsed firewall rules:")
    for rule in WindowsFirewallRuleParser.parse(text, separator='\t'):
        print(vars(rule))

Parsed firewall rules:
{'name': 'Foo1', 'remote_addresses': <__main__.AddressSet object at 0x00000200E9ABCAA0>, 'remote_ports': <__main__.PortSet object at 0x00000200E9ABCF20>, 'local_ports': <__main__.PortSet object at 0x00000200E9ABCF50>, 'protocol': <__main__.NetworkProtocol object at 0x00000200E9ABCEF0>, 'enabled': True, 'allow': True}
{'name': 'Bar1', 'remote_addresses': <__main__.AddressSet object at 0x00000200E9ABCBF0>, 'remote_ports': <__main__.PortSet object at 0x00000200E9ABD0A0>, 'local_ports': <__main__.PortSet object at 0x00000200E9ABD100>, 'protocol': <__main__.NetworkProtocol object at 0x00000200E9ABCB60>, 'enabled': True, 'allow': True}


In [2]:
# Python version of WindowsFirewallRuleParser.cs
# This assumes supporting classes like AddressSet, AddressRange, PortSet, PortRange, and NetworkProtocol are defined.

import ipaddress
import csv
from typing import List, Dict, Generator, Optional

class AddressRange:
    def __init__(self, low: str, high: str):
        self.low = ipaddress.IPv4Address(low)
        self.high = ipaddress.IPv4Address(high)

class AddressSet:
    def __init__(self, contains_all: bool = False, ranges: Optional[List[AddressRange]] = None):
        self.contains_all = contains_all
        self.ranges = ranges or []

class PortRange:
    def __init__(self, low: int, high: int):
        self.low = low
        self.high = high

class PortSet:
    def __init__(self, contains_all: bool = False, ranges: Optional[List[PortRange]] = None):
        self.contains_all = contains_all
        self.ranges = ranges or []

class NetworkProtocol:
    def __init__(self, any_protocol: bool = False, protocol_number: int = 0):
        self.any = any_protocol
        self.protocol_number = protocol_number

    @staticmethod
    def try_get_protocol_number(name: str) -> Optional[int]:
        protocol_map = {"TCP": 6, "UDP": 17, "ICMP": 1}
        return protocol_map.get(name.upper())

class WindowsFirewallRule:
    def __init__(self, name: str, remote_addresses: AddressSet, remote_ports: PortSet,
                 local_ports: PortSet, protocol: NetworkProtocol, enabled: bool, allow: bool):
        self.name = name
        self.remote_addresses = remote_addresses
        self.remote_ports = remote_ports
        self.local_ports = local_ports
        self.protocol = protocol
        self.enabled = enabled
        self.allow = allow

    def __str__(self):
        return (
            f"Rule: {self.name}\n"
            f"  Enabled: {self.enabled}, Action: {'Allow' if self.allow else 'Block'}\n"
            f"  Protocol: {'Any' if self.protocol.any else self.protocol.protocol_number}\n"
            f"  Local Ports: {'Any' if self.local_ports.contains_all else [(r.low, r.high) for r in self.local_ports.ranges]}\n"
            f"  Remote Ports: {'Any' if self.remote_ports.contains_all else [(r.low, r.high) for r in self.remote_ports.ranges]}\n"
            f"  Remote Addresses: {'Any' if self.remote_addresses.contains_all else [(str(r.low), str(r.high)) for r in self.remote_addresses.ranges]}"
        )

class WindowsFirewallRuleParser:
    REQUIRED_HEADERS = [
        "Name", "Enabled", "Action", "Local Port",
        "Remote Address", "Remote Port", "Protocol"
    ]

    @staticmethod
    def parse(text: str, separator: str) -> Generator[WindowsFirewallRule, None, None]:
        reader = csv.reader(text.strip().splitlines(), delimiter=separator)
        header_line = next(reader)
        header_index = WindowsFirewallRuleParser.parse_header(header_line)

        for i, line in enumerate(reader):
            try:
                yield WindowsFirewallRuleParser.parse_record(header_index, line)
            except Exception as e:
                print(f"Skipping line {i + 2} - {e}")

    @staticmethod
    def parse_header(header_line: List[str]) -> Dict[str, int]:
        header_index = {h.strip(): i for i, h in enumerate(header_line)}
        missing = [h for h in WindowsFirewallRuleParser.REQUIRED_HEADERS if h not in header_index]
        if missing:
            raise ValueError(f"Missing required headers: {', '.join(missing)}")
        return header_index

    @staticmethod
    def parse_record(header_index: Dict[str, int], record: List[str]) -> WindowsFirewallRule:
        def get(field: str) -> str:
            return record[header_index[field]].strip()

        return WindowsFirewallRule(
            name=get("Name"),
            remote_addresses=WindowsFirewallRuleParser.parse_address_set(get("Remote Address")),
            remote_ports=WindowsFirewallRuleParser.parse_port_set(get("Remote Port")),
            local_ports=WindowsFirewallRuleParser.parse_port_set(get("Local Port")),
            protocol=WindowsFirewallRuleParser.parse_network_protocol(get("Protocol")),
            enabled=(get("Enabled") == "Yes"),
            allow=(get("Action") == "Allow")
        )

    @staticmethod
    def parse_address_set(text: str) -> AddressSet:
        if text == "Any":
            return AddressSet(contains_all=True)

        ranges = []
        for part in text.split(','):
            part = part.strip()
            if '-' in part:
                low, high = part.split('-')
                ranges.append(AddressRange(low.strip(), high.strip()))
            else:
                ranges.append(AddressRange(part, part))
        return AddressSet(contains_all=False, ranges=ranges)

    @staticmethod
    def parse_port_set(text: str) -> PortSet:
        if not text:
            raise ValueError("Port is empty")

        if text == "Any":
            return PortSet(contains_all=True)

        unsupported_macros = {
            "RPC Endpoint Mapper", "RPC Dynamic Ports",
            "IPHTTPS", "Edge Traversal", "PlayTo Discovery"
        }
        if text in unsupported_macros:
            raise ValueError(f"Unsupported port macro: {text}")

        ranges = []
        for part in text.split(','):
            part = part.strip()
            if '-' in part:
                low, high = part.split('-')
                ranges.append(PortRange(int(low.strip()), int(high.strip())))
            else:
                val = int(part)
                ranges.append(PortRange(val, val))
        return PortSet(contains_all=False, ranges=ranges)

    @staticmethod
    def parse_network_protocol(text: str) -> NetworkProtocol:
        if text == "Any":
            return NetworkProtocol(any_protocol=True)

        protocol_number = NetworkProtocol.try_get_protocol_number(text)
        if protocol_number is None:
            protocol_number = int(text)

        return NetworkProtocol(any_protocol=False, protocol_number=protocol_number)

# Example of taking input and parsing
if __name__ == '__main__':
    import sys
    
    filename = input("Enter the firewall dump file path: ")
    # separator = input("Enter the column separator (e.g., ',', '\\t'): ")

    with open(filename, 'r') as f:
        text = f.read()

    print("Parsed firewall rules:")
    for rule in WindowsFirewallRuleParser.parse(text, separator='\t'):
        print(rule)


Parsed firewall rules:
Rule: Foo1
  Enabled: True, Action: Allow
  Protocol: 17
  Local Ports: [(100, 100)]
  Remote Ports: [(100, 100)]
  Remote Addresses: [('10.3.141.0', '10.3.141.0')]
Rule: Bar1
  Enabled: True, Action: Allow
  Protocol: 6
  Local Ports: [(200, 200)]
  Remote Ports: [(200, 200)]
  Remote Addresses: [('10.3.141.0', '10.3.141.0')]


In [8]:
# This code checks equivalence of two parsed firewall rule sets using Z3
from z3 import *

class FirewallEquivalenceChecker:
    def __init__(self, rules1, rules2):
        self.rules1 = rules1
        self.rules2 = rules2

    def allows(self, rules, packet):
        allowed_exprs = []
        for rule in rules:
            if not rule.enabled:
                continue

            proto_cond = BoolVal(rule.protocol.any) if rule.protocol.any else packet['protocol'] == rule.protocol.protocol_number
            local_port_cond = self.port_set_expr(packet['local_port'], rule.local_ports)
            remote_port_cond = self.port_set_expr(packet['remote_port'], rule.remote_ports)
            remote_addr_cond = self.address_set_expr(packet['remote_ip'], rule.remote_addresses)
            match_expr = And(proto_cond, local_port_cond, remote_port_cond, remote_addr_cond)

            allowed_exprs.append(And(match_expr, BoolVal(rule.allow)))

        return Or(*allowed_exprs) if allowed_exprs else BoolVal(False)

    def port_set_expr(self, port_var, port_set):
        if port_set.contains_all:
            return BoolVal(True)
        return Or([And(port_var >= r.low, port_var <= r.high) for r in port_set.ranges])

    def address_set_expr(self, ip_var, addr_set):
        if addr_set.contains_all:
            return BoolVal(True)
        return Or([
            And(ip_var >= self.ip_to_int(r.low), ip_var <= self.ip_to_int(r.high))
            for r in addr_set.ranges
        ])

    def ip_to_int(self, ip):
        return int(ip)

    def check_equivalence(self):
        # ctx = Context()
        s = Solver()

        packet = {
            'protocol': Int('proto'),
            'local_port': Int('lport'),
            'remote_port': Int('rport'),
            'remote_ip': Int('rip')
        }

        f1_allows = self.allows(self.rules1, packet)
        f2_allows = self.allows(self.rules2, packet)

        inequivalence = Not(f1_allows == f2_allows)
        s.add(inequivalence)

        if s.check() == unsat:
            print("Firewalls are equivalent.")
        else:
            print("Firewalls are NOT equivalent.")
            m = s.model()
            print("Counterexample:")
            for k, v in packet.items():
                if k == 'remote_ip':
                    print(f"  {k}: {ipaddress.IPv4Address(m[v].as_long())}")
                else :
                    print(f"  {k}: {m[v]}")


if __name__ == '__main__':
    # from windows_firewall_parser import WindowsFirewallRuleParser

    sep = '\t'#input("Enter separator (e.g., ',' or '\\t'): ")
    file1 = input("Enter first firewall file: ")
    file2 = input("Enter second firewall file: ")

    with open(file1) as f1:
        rules1 = list(WindowsFirewallRuleParser.parse(f1.read(), sep))
    with open(file2) as f2:
        rules2 = list(WindowsFirewallRuleParser.parse(f2.read(), sep))

    checker = FirewallEquivalenceChecker(rules1, rules2)
    checker.check_equivalence()

Firewalls are equivalent.


In [3]:
from z3 import *
s = Solver()